In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import configparser
import os.path
from os import path
from importlib import reload
import wandb


creds_path_ar = ["../../credentials.ini","credentials.colab.ini"]
PATH_ROOT = ""
PATH_DATA = ""

for creds_path in creds_path_ar:
    if path.exists(creds_path):
        config_parser = configparser.ConfigParser()
        config_parser.read(creds_path)
        PATH_ROOT = config_parser['MAIN']["PATH_ROOT"]
        PATH_DATA = config_parser['MAIN']["PATH_DATA"]
        WANDB_enable = config_parser['MAIN']["WANDB_ENABLE"] == 'TRUE'
        ENV = config_parser['MAIN']["ENV"]
        break

if ENV=="COLAB":
  from google.colab import drive
  mount_path = '/content/gdrive/'
  drive.mount(mount_path)

In [3]:
cd {PATH_ROOT}

/home/shaul/workspace/GitHub/sota-mafat-radar


In [4]:
import os
import random
import pickle
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim


from sklearn.metrics import roc_auc_score, roc_curve, auc, accuracy_score
from matplotlib.colors import LinearSegmentedColormap
from termcolor import colored

from src.data import feat_data, get_data
from src.features import specto_feat
from src.models import arch_setup, base_base_model, alex_model, tcn_model
from tqdm import tqdm

# Set seed for reproducibility of results
seed_value = 0
os.environ['PYTHONHASHSEED']=str(seed_value)


random.seed(seed_value)
np.random.seed(seed_value)
torch.manual_seed(0)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

if torch.cuda.is_available():
    device = torch.device('cuda:0')
else:
    device = torch.device('cpu:0')

In [5]:
config = dict()
config['num_tracks'] = 4
config['val_ratio'] = 6
config['shift_segment'] = np.arange(1,32)
config['get_shifts'] = False
config['get_horizontal_flip'] = False
config['get_vertical_flip'] = False

In [6]:
# train_x, train_y, val_x, val_y = get_data.classic_trainval(PATH_DATA)

In [7]:
train_path = 'MAFAT RADAR Challenge - Training Set V1'
training_dict = get_data.load_data(train_path, PATH_DATA)

#split Tracks here to only do augmentation on Train set
train_dict, val_dict = get_data.split_train_val_as_df(training_dict,ratio= 6)

In [8]:
synth_data = get_data.load_data(PATH_ROOT + PATH_DATA + 'MAFAT RADAR Challenge - Auxiliary Synthetic Set V2')
synth_data['segment_id'] = np.array(synth_data['segment_id'].tolist()) - 2000000

In [59]:
[x['iq_sweep_burst'] for x in train_dict if x['segment_id'] in id_train]

TypeError: string indices must be integers

In [36]:
id_train = list(set(synth_data['segment_id']).intersection(set(train_dict['segment_id'])))
id_train.sort()
synth_train_xhat = synth_data['iq_sweep_burst'][id_train]
synth_train_x = train_dict['iq_sweep_burst'][id_train]

IndexError: index 6347 is out of bounds for axis 0 with size 6347

In [51]:
train_dict['segment_id'].shape

(6347,)

In [46]:
max(list(set(train_dict['segment_id']).intersection(set(synth_data['segment_id']))) )

6567

In [22]:
id_val = list(set(synth_data['segment_id']).intersection(set(val_dict['segment_id'])))
id_val.sort()
synth_val = synth_data['iq_sweep_burst'][id_val]

In [23]:
class DS(Dataset):
    def __init__(self,noisy_df,clean_df):
        super().__init__()
        self.noisy_df = noisy_df
        self.clean_df = clean_df

    def __len__(self):
        return self.noisy_df.shape[0]
        
    def __getitem__(self, idx):
        x = self.noisy_df[idx]
        x_hat = self.clean_df[idx]
        return x, x_hat

In [24]:
train_set = DS(synth_train,train_dict['iq_sweep_burst'])
val_set = DS(synth_val, val_dict['iq_sweep_burst'])
train_loader = DataLoader(dataset= train_set, batch_size = 32, shuffle = True, num_workers = 2)
val_loader = DataLoader(dataset= val_set, batch_size = 32, shuffle = True, num_workers = 2)

In [25]:
'''
# DeConv1
x = Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = UpSampling2D((2, 2))(x)

# DeConv2
x = Conv2D(8, (3, 3), activation='relu', padding='same')(x)
x = UpSampling2D((2, 2))(x)

# Deconv3
x = Conv2D(16, (3, 3), activation='relu')(x)
x = UpSampling2D((2, 2))(x)
decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)
'''

# class autoencoder(nn.Module):
    # def __init__(self):
    #     super(autoencoder, self).__init__()
    #     self.encoder = nn.Sequential(
    #         nn.Conv2d(1,16, kernel_size=(3,3),padding=(2,2)),
    #         nn.ReLU(True),
    #         nn.MaxPool2d(kernel_size=(2,2),padding=(2,2)),
    #         nn.Conv2d(32,64, kernel_size=(5,5),padding=(2,2)),
    #         nn.ReLU(True),
    #         nn.MaxPool2d(kernel_size=(2,2),padding=(2,2)),
    #         nn.Conv2d(64,128, kernel_size=(5,5),padding=(2,2)),
    #         nn.ReLU(True),
    #         nn.MaxPool2d(kernel_size=(2,2),padding=(2,2)),
    #         nn.Linear(128*32*128, 64),
    #         nn.ReLU(True))
    #     self.decoder = nn.Sequential(
    #         nn.ConvTranspose2d(1,16,3,2),
    #         nn.ReLU(True),
    #         nn.ConvTranspose2d(16,32,3,2),
    #         nn.ReLU(True),
    #         nn.ConvTranspose2d(32,64,3,2),
    #         nn.ReLU(True),
    #         nn.ConvTranspose2d(64,1,2,2),
    #         nn.Sigmoid())

    # def forward(self, x):
    #     x = self.encoder(x)
    #     x = self.decoder(x)
    #     return x

class ConvAutoencoder(nn.Module):
    def __init__(self):
        super(ConvAutoencoder, self).__init__()
       
        #Encoder
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)  
        self.conv2 = nn.Conv2d(16, 4, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
       
        #Decoder
        self.t_conv1 = nn.ConvTranspose2d(4, 16, 2, stride=2)
        self.t_conv2 = nn.ConvTranspose2d(16, 1, 2, stride=2)


    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.t_conv1(x))
        x = F.sigmoid(self.t_conv2(x))
              
        return x

In [26]:
model = ConvAutoencoder().to(device)
criterion=nn.MSELoss()
optimizer=optim.SGD(model.parameters(),lr=1e-3)

In [32]:
epochs=20
l=len(train_loader)
losslist=list()
epochloss=0
running_loss=0
for epoch in range(epochs):
  
  print("Entering Epoch: ",epoch)
  for dirty,clean in tqdm((train_loader)):
    
    
    # dirty=dirty.view(dirty.size(0),-1).type(torch.FloatTensor)
    # clean=clean.view(clean.size(0),-1).type(torch.FloatTensor)
    dirty = torch.unsqueeze(dirty,1)
    clean = torch.unsqueeze(clean,1)
    dirty,clean=dirty.to(device),clean.to(device)
    
    #-----------------Forward Pass----------------------
    output=model(dirty)
    loss=criterion(output,clean)
    #-----------------Backward Pass---------------------
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    running_loss+=loss.item()
    epochloss+=loss.item()
  #-----------------Log-------------------------------
  losslist.append(running_loss/l)
  running_loss=0
  print("======> epoch: {}/{}, Loss:{}".format(epoch,epochs,loss.item()))

  0%|          | 0/71 [00:00<?, ?it/s]


TypeError: Caught TypeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/shaul/workspace/GitHub/sota-mafat-radar/venv_mafat/lib/python3.6/site-packages/torch/utils/data/_utils/worker.py", line 178, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/shaul/workspace/GitHub/sota-mafat-radar/venv_mafat/lib/python3.6/site-packages/torch/utils/data/_utils/fetch.py", line 47, in fetch
    return self.collate_fn(data)
  File "/home/shaul/workspace/GitHub/sota-mafat-radar/venv_mafat/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 79, in default_collate
    return [default_collate(samples) for samples in transposed]
  File "/home/shaul/workspace/GitHub/sota-mafat-radar/venv_mafat/lib/python3.6/site-packages/torch/utils/data/_utils/collate.py", line 79, in <listcomp>
    return [default_collate(samples) for samples in transposed]
  File "/home/shaul/workspace/GitHub/sota-mafat-radar/venv_mafat/lib/python3.6/site-package

In [15]:
# !pip install torch==1.5.1

  Using cached torch-1.5.1-cp36-cp36m-manylinux1_x86_64.whl (753.2 MB)
ERROR: torchvision 0.7.0 has requirement torch==1.6.0, but you'll have torch 1.5.1 which is incompatible.
  Attempting uninstall: torch
    Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
You should consider upgrading via the '/home/shaul/workspace/GitHub/sota-mafat-radar/venv_mafat/bin/python -m pip install --upgrade pip' command.


# NO MANDS LAND

array([47, 64, 67, 67,  9, 83, 21, 36, 87, 70, 88, 88, 12, 58, 65])

In [51]:
dict1 = dict()
dict2 = dict()
dict1['id'] = list(range(10))
dict1['val'] = np.random.rand(10,12)
dict2['id'] = list(range(5,15))
dict2['val'] = np.random.randint(0,100,10)

In [57]:
def combine_dicts(train_dict, synth_dict):
    td = dict()
    sd = dict()

    seg_id = train_dict['segment_id']
    iq_sb = train_dict['iq_sweep_burst']

    for a,b in zip(seg_id,iq_sb):
        td[a] = b

    seg_id = synth_dict['segment_id']
    iq_sb = synth_dict['iq_sweep_burst']

    for a,b in zip(seg_id,iq_sb):
        sd[a] = b

    comb_dict = dict()

    for key1,val1 in td.items():
        for key2, val2 in sd.items():
            if key1 == key2:
                comb_dict[key1] = dict()
                comb_dict[key1]['train_iq'] = val1
                comb_dict[key1]['synth_iq'] = val2
    
    return comb_dict

In [59]:
cd = combine_dicts(train_dict,synth_data)

In [63]:
cd['segment_id']

6567

In [65]:
td = dict()
sd = dict()

seg_id = train_dict['segment_id']
iq_sb = train_dict['iq_sweep_burst']

for a,b in zip(seg_id,iq_sb):
    td[a] = b

seg_id = synth_data['segment_id']
iq_sb = synth_data['iq_sweep_burst']

for a,b in zip(seg_id,iq_sb):
    sd[a] = b

In [75]:
comb_dict = dict()

i = 0

for key1,val1 in td.items():
    for key2, val2 in sd.items():
        if key1 == key2:
            i = i +1
            comb_dict[key1] = dict()
            comb_dict[key1]['train_iq'] = val1
            comb_dict[key1]['synth_iq'] = val2
            if i == 2:
                print(comb_dict)
                break
return comb_dict

NameError: name 'i' is not defined